In [0]:
# /FileStore/tables/dataset_Facebook_cos.csv

data=spark.read.csv("/FileStore/tables/dataset_Facebook_cos.csv",header=True,inferSchema=True)
data.show()

+----------------+------+--------+----------+------------+---------+----+-------------------------+-------------------------------+----------------------+-----------------------+--------------------------+------------------------------------------------------------+------------------------------------------------+-------------------------------------------------------------------+-------+----+-----+------------------+
|Page total likes|  Type|Category|Post Month|Post Weekday|Post Hour|Paid|Lifetime Post Total Reach|Lifetime Post Total Impressions|Lifetime Engaged Users|Lifetime Post Consumers|Lifetime Post Consumptions|Lifetime Post Impressions by people who have liked your Page|Lifetime Post reach by people who like your Page|Lifetime People who have liked your Page and engaged with your post|comment|like|share|Total Interactions|
+----------------+------+--------+----------+------------+---------+----+-------------------------+-------------------------------+----------------------+--

In [0]:
#1. The total number of posts made

print("total number of posts made are: ",data.select("Type").count())


total number of posts made are:  500


In [0]:
#2. The percentage of the growth or decline of the page, in terms of likes (subscriptions on the page), from the first post to the latest post

#data.select("Post Month","Lifetime Post reach by people who like your Page").show()

from pyspark.sql.window import Window

from pyspark.sql.functions import *
data2=data.groupBy("Post Month").agg(sum("Lifetime Post reach by people who like your Page").alias("post_reach_subscription"))

data2=data2.withColumn("post_reach_subscription_percent",col('post_reach_subscription')/sum('post_reach_subscription').over(Window.partitionBy())*100).orderBy("Post Month")
data2=data2.select("*",round("post_reach_subscription_percent",2)).drop("post_reach_subscription_percent")
display(data2)

Post Month,post_reach_subscription,"round(post_reach_subscription_percent, 2)"
1,135444,4.11
2,227958,6.92
3,233150,7.08
4,417592,12.68
5,243374,7.39
6,332009,10.08
7,434163,13.19
8,278027,8.44
9,176227,5.35
10,286687,8.71


Databricks visualization. Run in Databricks to view.

In [0]:
#3. Which month, on average, has the highest number of post interactions?

from pyspark.sql.functions import *
print("highest number of post interactions are in the month : ")
data.groupBy("Post Month").agg(avg(col("Total Interactions")).alias("avg_total_Interactions")).orderBy(desc("avg_total_Interactions")).limit(1).show()




highest number of post interactions are in the month : 
+----------+----------------------+
|Post Month|avg_total_Interactions|
+----------+----------------------+
|         7|                 328.5|
+----------+----------------------+



In [0]:
#4. Which day of the week, on average, has the highest number of post interactions?

print("highest number of post interactions are in day of the week : ")
data.groupBy("Post Weekday").agg(avg(col("Total Interactions")).alias("avg_total_Interactions")).orderBy(desc("avg_total_Interactions")).limit(1).show()

highest number of post interactions are in day of the week : 
+------------+----------------------+
|Post Weekday|avg_total_Interactions|
+------------+----------------------+
|           3|            287.765625|
+------------+----------------------+



In [0]:
#5. Which hour of the day, on average, has the highest number of post interactions?


print("highest number of post interactions per hour of the day are : ")
data.groupBy("Post Hour").agg(avg(col("Total Interactions")).alias("avg_total_Interactions")).orderBy(desc("avg_total_Interactions")).limit(1).show()

highest number of post interactions per hour of the day are : 
+---------+----------------------+
|Post Hour|avg_total_Interactions|
+---------+----------------------+
|        5|     684.3076923076923|
+---------+----------------------+



In [0]:
"""6. Determine if paid (promoted) posts have a higher correlation with a large number 
of post shares when compared to the post shares of organic (non-promoted) posts. 
This is to determine the commercial viability of investing in paid posts for 
promoting cosmetic products. Answer with either a Yes or a No, and provide the 
methodology of how you reached your conclusion"""



data.filter(col("Paid")==1).groupBy("Paid").agg(avg("share")).show()
data.filter(col("Paid")==0).groupBy("Paid").agg(avg("share")).show()
print("My Answer is 'YES',The average number of shares are more for Paid Posts and reach is more for paid posts....commercial viability of investing in paid posts for promoting cosmetic products is necessary")

+----+-----------------+
|Paid|       avg(share)|
+----+-----------------+
|   1|32.50359712230216|
+----+-----------------+

+----+------------------+
|Paid|        avg(share)|
+----+------------------+
|   0|25.219101123595507|
+----+------------------+

My Answer is 'YES',The average number of shares are more for Paid Posts and reach is more for paid posts....commercial viability of investing in paid posts for promoting cosmetic products is necessary


In [0]:
#7. Which post type (photo, video, status, or link) is the most attractive to people who have subscribed to your page (people who have liked the page)?

print("The type of post which is most attractive to people who have subscribed to page")

data.groupBy("Type").agg(sum("Lifetime People who have liked your Page and engaged with your post")).show()

The type of post which is most attractive to people who have subscribed to page
+------+------------------------------------------------------------------------+
|  Type|sum(Lifetime People who have liked your Page and engaged with your post)|
+------+------------------------------------------------------------------------+
| Video|                                                                    6856|
|  Link|                                                                    4632|
| Photo|                                                                  216112|
|Status|                                                                   77393|
+------+------------------------------------------------------------------------+



In [0]:
# 8. Which hour of the day is ideal for posting photographic content? Arrange the hours of the day according to the order of the Lifetime Post Impressions column?


print("Hour of the day is ideal for posting photographic content is :")

data.filter(col("Type")=="Photo").groupBy("Post Hour").agg(sum("Lifetime Post Total Impressions").alias("Lifetime Post Impressions")).orderBy(desc("Lifetime Post Impressions")).limit(1).show()

Hour of the day is ideal for posting photographic content is :
+---------+-------------------------+
|Post Hour|Lifetime Post Impressions|
+---------+-------------------------+
|       13|                  2389483|
+---------+-------------------------+



In [0]:
#9. Create an additional column with the name Likes-to-comment Ratio, with the column values having the equation:

ratio=data["like"]/data["comment"]
data2=data.withColumn("Likes-to-comment Ratio",round(ratio,2))
data2.display()


Page total likes,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions,Likes-to-comment Ratio
139441,Photo,2,12,4,3,0,2752,5091,178,109,159,3078,1640,119,4,79,17,100,19.75
139441,Status,2,12,3,10,0,10460,19057,1457,1361,1674,11710,6112,1108,5,130,29,164,26.0
139441,Photo,3,12,3,3,0,2413,4373,177,113,154,2812,1503,132,0,66,14,80,null
139441,Photo,2,12,2,10,1,50128,87991,2211,790,1119,61027,32048,1386,58,1572,147,1777,27.1
139441,Photo,2,12,2,3,0,7244,13594,671,410,580,6228,3200,396,19,325,49,393,17.11
139441,Status,2,12,1,9,0,10472,20849,1191,1073,1389,16034,7852,1016,1,152,33,186,152.0
139441,Photo,3,12,1,3,1,11692,19479,481,265,364,15432,9328,379,3,249,27,279,83.0
139441,Photo,3,12,7,9,1,13720,24137,537,232,305,19728,11056,422,0,325,14,339,null
139441,Status,2,12,7,3,0,11844,22538,1530,1407,1692,15220,7912,1250,0,161,31,192,null
139441,Photo,3,12,6,10,0,4694,8668,280,183,250,4309,2324,199,3,113,26,142,37.67


In [0]:
#10. Arrange post categories (1,2,3) in the descending order of the reach that they can accumulate on average

#data.show()
data.groupBy("Category").agg(avg(col("Lifetime Post Total Reach")).alias("avg_reach")).orderBy(desc("avg_reach")).show()


+--------+------------------+
|Category|         avg_reach|
+--------+------------------+
|       1|18320.981395348837|
|       3|11162.154838709677|
|       2| 9865.653846153846|
+--------+------------------+



In [0]:
"""11. Determine the standard deviation of the average post reach for each of the day 
hours. This is to determine if the time of the day is an ideal criterion to identify 
when to create posts"""

from pyspark.sql.functions import *
data2=data.groupBy("Post Hour").agg(stddev("Lifetime Post Total Reach").alias("reach_for_dayHour")).orderBy("Post Hour").show()
#data2.groupBy("Post Hour").agg(stddev("avg_reach_for_dayHour")).show()

+---------+------------------+
|Post Hour| reach_for_dayHour|
+---------+------------------+
|        1|1668.8739467077794|
|        2|28964.266723847893|
|        3| 20062.48506691216|
|        4|16179.952578173177|
|        5| 48900.71604032409|
|        6| 19384.93338953391|
|        7| 14535.58762935882|
|        8|  2586.13118676364|
|        9|12813.220059900956|
|       10|22449.330739753404|
|       11| 9433.434571988919|
|       12|16929.348611911955|
|       13|31605.113444715214|
|       14| 41999.64255830805|
|       15| 1875.010826635409|
|       16|              null|
|       17|6172.8230170643965|
|       18|3004.5780069753555|
|       19|              null|
|       20|              null|
+---------+------------------+
only showing top 20 rows



In [0]:
#12. Is there any correlation between the number of post consumptions and the total interactions on the post?

#data.select("Lifetime Post Consumptions","Total Interactions").show()

data.stat.corr("Lifetime Post Consumptions", "Total Interactions")

Out[18]: 0.2380821957904635

In [0]:
#13. Determine the two best days in a week to create posts, when people are extremely active on social media, based on the data that you have

print("The days in which reach is more is considered as best days to create posts :  ")
data.groupBy("Post Weekday").agg(sum("Lifetime Post Total Impressions").alias("post appeared in people’s feeds")).orderBy(desc("post appeared in people’s feeds")).limit(2).show()



The days in which reach is more is considered as best days to create posts :  
+------------+-------------------------------+
|Post Weekday|post appeared in people’s feeds|
+------------+-------------------------------+
|           4|                        3279613|
|           6|                        2302438|
+------------+-------------------------------+

